# Dependencies

In [3]:
import pandas as pd
from langchain.document_loaders import PyPDFDirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter 
from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from pinecone import Pinecone, ServerlessSpec
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate 
from dotenv import load_dotenv
load_dotenv()
import os
import time

# Data Loading and Cleaning
Here, we will be loading all the content as stored by a user in the form of a csv. Now, the each row will be converted as an embedding and stored in the vectorDB. We can go ahead and load the data and see the first few rows of the data.

In [60]:
data = pd.read_csv("./data/Demo.csv")
data

,_id,link,type,title,description,tags[0],tags[1],userID,__v
0,67559f6ee33e21842371fd7b,https://openai.com/index/introducing-chatgpt-pro/,text,GPT Pro - $200/month,NaN,development,AI,67559e7229c6ffb06be7a883,0
1,67559fe0e33ecc79f41ea880,https://www.youtube.com/watch?v=zKd_kxcxGbo,video,The rise of mediocrity,NaN,development,motivation,67559e7229c6ffb06be7a883,0
2,6755a18be33e21842371fd91,https://medium.com/gopenai/unlocking-domain-ex...,text,RAG - Intro,NaN,AI,development,67559e7229c6ffb06be7a883,0


Now, we want to merge all columns except "_id", "userID" and "description" into a single column. This is because we want to convert all the content into a single embedding.

In [64]:
data[data.columns.difference(['_id', 'userID', 'description','__v'])].apply(lambda x: ' : '.join(x.dropna().astype(str)), axis=1)[0]

'https://openai.com/index/introducing-chatgpt-pro/ : development : AI : GPT Pro - $200/month : text'

In [58]:
data_mod = data[data.columns.difference(['_id', 'userID', 'description','__v'])]
data_mod = data.apply(lambda x: ' : '.join(x.dropna().astype(str)), axis=1)

In [59]:
data

,_id,link,type,title,description,tags[0],tags[1],userID,__v
0,67559f6ee33e21842371fd7b,https://openai.com/index/introducing-chatgpt-pro/,text,GPT Pro - $200/month,NaN,development,AI,67559e7229c6ffb06be7a883,0
1,67559fe0e33ecc79f41ea880,https://www.youtube.com/watch?v=zKd_kxcxGbo,video,The rise of mediocrity,NaN,development,motivation,67559e7229c6ffb06be7a883,0
2,6755a18be33e21842371fd91,https://medium.com/gopenai/unlocking-domain-ex...,text,RAG - Intro,NaN,AI,development,67559e7229c6ffb06be7a883,0
